<a href="https://colab.research.google.com/github/RubanZ/ppa-for-da/blob/master/students/%D0%A1%D1%82%D1%80%D0%BE%D0%BA%D0%BE%D0%B2_%D0%94%D0%BC%D0%B8%D1%82%D1%80%D0%B8%D0%B9/Natasha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Предобработка текстовых данных с помощью фреймворка Natasha

Natasha решает базовые задачи NLP (natural language processing) для русского языка: токенизация, сегментация предложений, встраивание слов, морфология, лемматизация, нормализация фраз, синтаксический анализ, работа с NER, извлечение фактов. 

[Natasha](https://natasha.github.io/ner/) объединяет библиотеки проекта Natasha под одним удобным API:

* [Razdel](https://natasha.github.io/razdel/) - токенизация, сегментация предложений для русского языка

* [Navec](https://natasha.github.io/navec/) - вектора русских слов

* [Slovnet](https://github.com/natasha/slovnet) - библиотека с методами глубокого обучения для русского NLP, компактные модели для русской морфологии, синтаксиса и NER.

* [Yargy](https://github.com/natasha/yargy) - извлечение фактов на основе правил, аналогичное синтаксическому анализатору Tomita.

Рассмотрим примеры использования

In [1]:
 !pip install natasha

     ,████████████████████████████████, 34.4 MB 149 kB/s 
     ,████████████████████████████████, 49 kB 6.1 MB/s 
     ,████████████████████████████████, 41 kB 112 kB/s 
     ,████████████████████████████████, 55 kB 3.8 MB/s 
     ,████████████████████████████████, 8.2 MB 40.7 MB/s 
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26119 sha256=986ad8dd6e36e4e520cb3c2168d1bdbf595961aef6727aa41a7a7a0e3707eb67
  Stored in directory: /root/.cache/pip/wheels/16/85/bd/1001cbb46dcfb71c2001cd7401c6fb250392f22a81ce3722f7
Successfully built intervaltree
  Attempting uninstall: intervaltree
    Found existing installation: intervaltree 2.1.0
    Uninstalling intervaltree-2.1.0:
      Successfully uninstalled intervaltree-2.1.0


In [2]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

In [3]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

text = '''Студенты и сотрудники лаборатории Машинного обучения Университета ИТМО разработали библиотеку для Python, которая решает ключевую задачу машинного обучения.

Расскажем, почему появился этот инструмент и что он умеет.
Нехватка алгоритмов

Одна из ключевых задач машинного обучения — снижение размерности данных. Дата-саентисты сокращают число переменных, вычленяя среди них значения, наибольшим образом влияющие на результат. После этой операции модель машинного обучения требует меньше памяти, работает быстрее и качественнее. Пример ниже показывает, что исключение дублирующих признаков увеличивает точность классификации с 0,903 до 0,943.

Существует два подхода к уменьшению размерности — конструирование и выбор признаков. В областях вроде биоинформатики и медицины чаще используют последний, так как он позволяет выделить значимые признаки с сохранением семантики, то есть не меняет исходный смысл признаков. Однако в самых распространенных библиотеках машинного обучения на Python — scikit-learn, pytorch, keras, tensorflow — нет полноценного набора методов выбора признаков.

Для решения этой проблемы студенты и аспиранты Университета ИТМО разработали открытую библиотеку — ITMO_FS. Над ней трудится команда под руководством Ивана Сметанникова, доцента факультета информационных технологий и программирования, заместителя заведующего лабораторией Машинного обучения. Ведущий разработчик — Никита Пильненьский, закончивший магистратуру «Машинное обучение и анализ данных». Теперь он поступает в аспирантуру.

«За последние несколько лет в нашу лабораторию приходили запросы на решение задач, для которых не подходил стандартный инструментарий. Например, нам требовались ансамблирующие алгоритмы на основе объединения фильтров, или алгоритмы, учитывающие наличие заранее известных (экспертно-размеченных) значимых признаков.

Посмотрев на существующие решения, мы пришли к выводу, что они не только не содержат необходимые нам инструменты, но и не являются достаточно гибкими для их возможной мягкой интеграции. В контексте того, что среди таких библиотек конкуренция слаба, мы решили сделать свою библиотеку, исправляющую большинство недостатков».

— Иван Сметанников
'''
doc = Doc(text)

# Разделение

После сегментации можно работать как с отдельными токенами, так и с целыми предложениями (которые в себе также содержат токены)

In [9]:
doc.segment(segmenter)
doc.tokens[:5]

[DocToken(stop=8, text='Студенты'),
 DocToken(start=9, stop=10, text='и'),
 DocToken(start=11, stop=21, text='сотрудники'),
 DocToken(start=22, stop=33, text='лаборатории'),
 DocToken(start=34, stop=43, text='Машинного')]

In [8]:
doc.sents[:5]

[DocSent(stop=156, text='Студенты и сотрудники лаборатории Машинного обуче..., tokens=[...]),
 DocSent(start=158, stop=216, text='Расскажем, почему появился этот инструмент и что ..., tokens=[...]),
 DocSent(start=217, stop=310, text='Нехватка алгоритмов\n\nОдна из ключевых задач маш..., tokens=[...]),
 DocSent(start=311, stop=424, text='Дата-саентисты сокращают число переменных, вычлен..., tokens=[...]),
 DocSent(start=425, stop=526, text='После этой операции модель машинного обучения тре..., tokens=[...])]

# Морфология

Библиотека умеет проводить морфологический анализ, можно работать снова с каждым токеном. Также уже есть встроенный "красивый" вывод признаков

Зависит от шага с разделением и дополняет уже существующую информацию о токенах

In [11]:
doc.tag_morph(morph_tagger)
doc.tokens[:5]

[DocToken(stop=8, text='Студенты', pos='NOUN', feats=<Anim,Nom,Masc,Plur>),
 DocToken(start=9, stop=10, text='и', pos='CCONJ'),
 DocToken(start=11, stop=21, text='сотрудники', pos='NOUN', feats=<Anim,Nom,Masc,Plur>),
 DocToken(start=22, stop=33, text='лаборатории', pos='NOUN', feats=<Inan,Gen,Fem,Sing>),
 DocToken(start=34, stop=43, text='Машинного', pos='ADJ', feats=<Gen,Pos,Neut,Sing>)]

In [12]:
doc.sents[0].morph.print()

            Студенты NOUN,Animacy=Anim,Case=Nom,Gender=Masc,Number=Plur
                   и CCONJ
          сотрудники NOUN,Animacy=Anim,Case=Nom,Gender=Masc,Number=Plur
         лаборатории NOUN,Animacy=Inan,Case=Gen,Gender=Fem,Number=Sing
           Машинного ADJ,Case=Gen,Degree=Pos,Gender=Neut,Number=Sing
            обучения NOUN,Animacy=Inan,Case=Gen,Gender=Neut,Number=Sing
        Университета PROPN,Animacy=Inan,Case=Gen,Gender=Masc,Number=Sing
                ИТМО PROPN,Animacy=Inan,Case=Gen,Gender=Masc,Number=Sing
         разработали VERB,Aspect=Perf,Mood=Ind,Number=Plur,Tense=Past,VerbForm=Fin,Voice=Act
          библиотеку NOUN,Animacy=Inan,Case=Acc,Gender=Fem,Number=Sing
                 для ADP
              Python PROPN,Foreign=Yes
                   , PUNCT
             которая PRON,Case=Nom,Gender=Fem,Number=Sing
              решает VERB,Aspect=Imp,Mood=Ind,Number=Sing,Person=3,Tense=Pres,VerbForm=Fin,Voice=Act
            ключевую ADJ,Case=Acc,Degree=Pos,Gender=Fem,N

# Лемматизация

Зависит от шага с морфологическим разбором и дополняет уже существующую информацию о токенах

Выведем первые 15 примеров приведения слова к его нормальной форме

In [13]:
for token in doc.tokens:
    token.lemmatize(morph_vocab)
    
doc.tokens[:5]

[DocToken(stop=8, text='Студенты', pos='NOUN', feats=<Anim,Nom,Masc,Plur>, lemma='студент'),
 DocToken(start=9, stop=10, text='и', pos='CCONJ', lemma='и'),
 DocToken(start=11, stop=21, text='сотрудники', pos='NOUN', feats=<Anim,Nom,Masc,Plur>, lemma='сотрудник'),
 DocToken(start=22, stop=33, text='лаборатории', pos='NOUN', feats=<Inan,Gen,Fem,Sing>, lemma='лаборатория'),
 DocToken(start=34, stop=43, text='Машинного', pos='ADJ', feats=<Gen,Pos,Neut,Sing>, lemma='машинный')]

In [14]:
{_.text: _.lemma for _ in doc.tokens[:15]}

{',': ',',
 'Python': 'python',
 'ИТМО': 'итмо',
 'Машинного': 'машинный',
 'Студенты': 'студент',
 'Университета': 'университет',
 'библиотеку': 'библиотека',
 'для': 'для',
 'и': 'и',
 'которая': 'который',
 'лаборатории': 'лаборатория',
 'обучения': 'обучение',
 'разработали': 'разработать',
 'решает': 'решать',
 'сотрудники': 'сотрудник'}

# Синтаксический разбор

Зависит от шага с разделением на токены, а именно использует информацию о предложениях

In [15]:
doc.parse_syntax(syntax_parser)
doc.tokens[:5]


[DocToken(stop=8, text='Студенты', id='1_1', head_id='1_9', rel='nsubj', pos='NOUN', feats=<Anim,Nom,Masc,Plur>, lemma='студент'),
 DocToken(start=9, stop=10, text='и', id='1_2', head_id='1_3', rel='cc', pos='CCONJ', lemma='и'),
 DocToken(start=11, stop=21, text='сотрудники', id='1_3', head_id='1_1', rel='conj', pos='NOUN', feats=<Anim,Nom,Masc,Plur>, lemma='сотрудник'),
 DocToken(start=22, stop=33, text='лаборатории', id='1_4', head_id='1_3', rel='nmod', pos='NOUN', feats=<Inan,Gen,Fem,Sing>, lemma='лаборатория'),
 DocToken(start=34, stop=43, text='Машинного', id='1_5', head_id='1_6', rel='amod', pos='ADJ', feats=<Gen,Pos,Neut,Sing>, lemma='машинный')]

In [16]:
doc.sents[0].syntax.print()

  ┌────────► Студенты     nsubj
  │       ┌► и            cc
  │     ┌─└─ сотрудники   
  │ ┌─┌─└──► лаборатории  nmod
  │ │ │   ┌► Машинного    amod
  │ │ └──►└─ обучения     nmod
  │ └────►┌─ Университета nmod
  │       └► ИТМО         nmod
┌─└───────┌─ разработали  
│     ┌─┌─└► библиотеку   obj
│     │ │ ┌► для          case
│     │ └►└─ Python       nmod
│     │ ┌──► ,            punct
│     │ │ ┌► которая      nsubj
│     └►└─└─ решает       acl:relcl
│     │   ┌► ключевую     amod
│   ┌─└──►└─ задачу       obj
│   │     ┌► машинного    amod
│   └────►└─ обучения     nmod
└──────────► .            punct


# Распознование именованных сущностей

Зависит от шага с разделением на токены

In [17]:
doc.tag_ner(ner_tagger)
doc.spans[:5]

[DocSpan(start=34, stop=52, type='ORG', text='Машинного обучения', tokens=[...]),
 DocSpan(start=53, stop=70, type='ORG', text='Университета ИТМО', tokens=[...]),
 DocSpan(start=1130, stop=1147, type='ORG', text='Университета ИТМО', tokens=[...]),
 DocSpan(start=1233, stop=1251, type='PER', text='Ивана Сметанникова', tokens=[...]),
 DocSpan(start=1355, stop=1373, type='ORG', text='Машинного обучения', tokens=[...])]

In [18]:
doc.ner.print()

Студенты и сотрудники лаборатории Машинного обучения Университета ИТМО
                                  ORG─────────────── ORG──────────────
 разработали библиотеку для Python, которая решает ключевую задачу 
машинного обучения.
Расскажем, почему появился этот инструмент и что он умеет.
Нехватка алгоритмов
Одна из ключевых задач машинного обучения — снижение размерности 
данных. Дата-саентисты сокращают число переменных, вычленяя среди них 
значения, наибольшим образом влияющие на результат. После этой 
операции модель машинного обучения требует меньше памяти, работает 
быстрее и качественнее. Пример ниже показывает, что исключение 
дублирующих признаков увеличивает точность классификации с 0,903 до 
0,943.
Существует два подхода к уменьшению размерности — конструирование и 
выбор признаков. В областях вроде биоинформатики и медицины чаще 
используют последний, так как он позволяет выделить значимые признаки 
с сохранением семантики, то есть не меняет исходный смысл признаков. 
Однако

Затем такие сущности можно нормализовать


In [19]:
for span in doc.spans:
   span.normalize(morph_vocab)
doc.spans[:5]

[DocSpan(start=34, stop=52, type='ORG', text='Машинного обучения', tokens=[...], normal='Машинное обучение'),
 DocSpan(start=53, stop=70, type='ORG', text='Университета ИТМО', tokens=[...], normal='Университет ИТМО'),
 DocSpan(start=1130, stop=1147, type='ORG', text='Университета ИТМО', tokens=[...], normal='Университет ИТМО'),
 DocSpan(start=1233, stop=1251, type='PER', text='Ивана Сметанникова', tokens=[...], normal='Иван Сметанников'),
 DocSpan(start=1355, stop=1373, type='ORG', text='Машинного обучения', tokens=[...], normal='Машинное обучение')]

In [20]:
{_.text: _.normal for _ in doc.spans if _.text != _.normal}

{'Ивана Сметанникова': 'Иван Сметанников',
 'Машинного обучения': 'Машинное обучение',
 'Университета ИТМО': 'Университет ИТМО'}

А также рзделить на составные чати (имя, фамилия и тд)

In [21]:
for span in doc.spans:
   if span.type == PER:
       span.extract_fact(names_extractor)

doc.spans[:5]

[DocSpan(start=34, stop=52, type='ORG', text='Машинного обучения', tokens=[...], normal='Машинное обучение'),
 DocSpan(start=53, stop=70, type='ORG', text='Университета ИТМО', tokens=[...], normal='Университет ИТМО'),
 DocSpan(start=1130, stop=1147, type='ORG', text='Университета ИТМО', tokens=[...], normal='Университет ИТМО'),
 DocSpan(start=1233, stop=1251, type='PER', text='Ивана Сметанникова', tokens=[...], normal='Иван Сметанников', fact=DocFact(slots=[...])),
 DocSpan(start=1355, stop=1373, type='ORG', text='Машинного обучения', tokens=[...], normal='Машинное обучение')]

In [22]:
{_.normal: _.fact.as_dict for _ in doc.spans if _.type == PER}

{'Иван Сметанников': {'first': 'Иван', 'last': 'Сметанников'},
 'Никита Пильненьский': {'first': 'Никита', 'last': 'Пильненьский'}}

# Источники

https://github.com/natasha

https://natasha.github.io

# Ревью

## Передреев Дмитрий

Туториал прошел. Добавил заголовок и ссылки на документацию модулей проекта - в них более подробно разобраны примеры использования

##Рубан Александр
Туториал прошел. Изменил ввыод на более читаемый